<a href="https://colab.research.google.com/github/Brandon-Bernal-Alarcon/Notas--R/blob/main/Tidyverse/Unir_tablas_con_dplyr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#Librerías
library(dplyr)
library(tidyr) # Para usar replace_na

**inner_join()**

Sirve para combinar tablas que continen una clave en común, permite fusionar dos tablas con los mismos valores clave

Si se ocupa directo es

inner_join(data1, data2, by = c(columna_data1, columna_data2))

cuando son distintas las columnas hay que especificar, sino basta con una cambia la columna de la data 1 por la de la data2

en caso de usar tuberías %>% se ocupa solo .data2



In [4]:
# Data frames de ejemplo
sets <- data.frame(
  set_num = c("001", "002", "003"),
  theme_id = c(1, 2, 3),
  name = c("Set A", "Set B", "Set C")
)

themes <- data.frame(
  id = c(1, 2, 3),
  name = c("Theme X", "Theme Y", "Theme Z")
)

# Inner join en theme_id e id
joined_data <- inner_join(sets, themes, by = c("theme_id" = "id"))
joined_data

#?inner_join


set_num,theme_id,name.x,name.y
<chr>,<dbl>,<chr>,<chr>
001,1,Set A,Theme X
002,2,Set B,Theme Y
003,3,Set C,Theme Z


Podemos personalizar nombres de columnas repetidas, si no lo hacemos aparece como .x o .y agregamos 'suffix = c("_x","_y")'


In [6]:
joined_data_custom <- inner_join(sets, themes, by = c("theme_id" = "id"), suffix = c("_set", "_theme"))
joined_data_custom

set_num,theme_id,name_set,name_theme
<chr>,<dbl>,<chr>,<chr>
001,1,Set A,Theme X
002,2,Set B,Theme Y
003,3,Set C,Theme Z


**relación uno-a-muchos**

En una relación uno-a-muchos, un solo registro en una tabla puede relacionarse con múltiples registros en otra tabla. Esto es común en bases de datos, como cuando se unen conjuntos de LEGO con sus inventarios. Cada conjunto puede tener múltiples versiones, resultando en más filas después de la unión.

Es decir ejemplo quieres unir el ID = 1 del data.1 con el ID = 2 del data.2 pero cuando vez, el data.2 tiene 5 filas con ID =1 entonces al unirlo tienes 5 filas asociadas

**Unir varias tablas**


In [13]:

# Tabla de clientes
clientes <- data.frame(
  cliente_id = c(1, 2, 3),
  nombre = c("Ana", "Luis", "Carlos")
)

# Tabla de órdenes (varias órdenes por cliente)
ordenes <- data.frame(
  orden_id = c(101, 102, 103, 104),
  cliente_id = c(1, 2, 1, 3),
  fecha = as.Date(c("2023-01-15", "2023-01-20", "2023-02-05", "2023-03-10"))
)

# Tabla de productos por orden (varios productos por orden)
productos_ordenados <- data.frame(
  orden_id = c(101, 101, 102, 103, 104, 104),
  producto = c("Laptop", "Mouse", "Tablet", "Monitor", "Teclado", "Mouse"),
  precio = c(12000, 300, 7000, 4000, 600, 350)
)

# Unimos las 3 tablas con inner_join
datos_completos <- clientes %>%
  inner_join(ordenes, by = "cliente_id") %>%
  inner_join(productos_ordenados, by = "orden_id")

# Resultado
print(datos_completos)

  cliente_id nombre orden_id      fecha producto precio
1          1    Ana      101 2023-01-15   Laptop  12000
2          1    Ana      101 2023-01-15    Mouse    300
3          1    Ana      103 2023-02-05  Monitor   4000
4          2   Luis      102 2023-01-20   Tablet   7000
5          3 Carlos      104 2023-03-10  Teclado    600
6          3 Carlos      104 2023-03-10    Mouse    350


**left_join() — Unión por la izquierda**

Mantiene todas las filas de la tabla izquierda, y completa con datos de la derecha (pone NA si no hay coincidencias). Si alguna ciudad de datos no aparece en ciudades_extra, la columna region será NA.



In [12]:
set.seed(123)
library(tibble)

datos <- tibble(
  id = 1:100,
  edad = sample(18:65, 100, replace = TRUE),
  sexo = sample(c("Hombre", "Mujer"), 100, replace = TRUE),
  ciudad = sample(c("CDMX", "Guadalajara", "Monterrey", "Puebla"), 100, replace = TRUE),
  ingreso_mensual = round(rnorm(100, mean = 15000, sd = 4000), 0),
  nivel_educativo = sample(c("Secundaria", "Preparatoria", "Universidad", "Posgrado"), 100, replace = TRUE)
)

In [14]:
ciudades_extra <- tibble(
  ciudad = c("CDMX", "Guadalajara", "Puebla", "Querétaro"),
  region = c("Centro", "Occidente", "Centro", "Bajío")
)

d <- datos %>%
  left_join(ciudades_extra, by = "ciudad")

head(d)

id,edad,sexo,ciudad,ingreso_mensual,nivel_educativo,region
<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>
1,48,Hombre,Monterrey,20490,Universidad,NA
2,32,Mujer,CDMX,12743,Posgrado,Centro
3,31,Hombre,Guadalajara,18881,Posgrado,Occidente
4,20,Mujer,Guadalajara,14925,Preparatoria,Occidente
5,59,Mujer,CDMX,16449,Secundaria,Centro
6,60,Mujer,Puebla,23045,Secundaria,Centro


**right_join() — Unión por la derecha**

Mantiene todas las filas de la tabla derecha, rellenando desde la izquierda.

Si hay ciudades en ciudades_extra que no están en datos, igual se conservan.


In [15]:
c <- ciudades_extra %>%
  right_join(datos, by = "ciudad")
head(c)

ciudad,region,id,edad,sexo,ingreso_mensual,nivel_educativo
<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<chr>
CDMX,Centro,2,32,Mujer,12743,Posgrado
CDMX,Centro,5,59,Mujer,16449,Secundaria
CDMX,Centro,11,44,Hombre,16257,Secundaria
CDMX,Centro,12,22,Mujer,22379,Universidad
CDMX,Centro,14,45,Mujer,23784,Preparatoria
CDMX,Centro,17,52,Mujer,11530,Universidad


**full_join() — Unión completa**

Combina todas las filas de ambas tablas, aunque no coincidan, Donde no haya coincidencia, aparecerán NA.



In [16]:
d <- datos %>%
  full_join(ciudades_extra, by = "ciudad")
head(d)

id,edad,sexo,ciudad,ingreso_mensual,nivel_educativo,region
<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>
1,48,Hombre,Monterrey,20490,Universidad,NA
2,32,Mujer,CDMX,12743,Posgrado,Centro
3,31,Hombre,Guadalajara,18881,Posgrado,Occidente
4,20,Mujer,Guadalajara,14925,Preparatoria,Occidente
5,59,Mujer,CDMX,16449,Secundaria,Centro
6,60,Mujer,Puebla,23045,Secundaria,Centro


**semi_join() — Filtrado por coincidencia**

Devuelve solo filas de la tabla izquierda que tienen coincidencia en la derecha.


In [19]:
ciudades_filtradas <- tibble(
  ciudad = c("CDMX", "Puebla")
)

d <- datos %>%
  semi_join(ciudades_filtradas, by = "ciudad")
head(d) #Solo conserva personas que viven en CDMX o Puebla.

id,edad,sexo,ciudad,ingreso_mensual,nivel_educativo
<int>,<int>,<chr>,<chr>,<dbl>,<chr>
2,32,Mujer,CDMX,12743,Posgrado
5,59,Mujer,CDMX,16449,Secundaria
6,60,Mujer,Puebla,23045,Secundaria
7,54,Mujer,Puebla,10288,Preparatoria
8,31,Mujer,Puebla,11979,Preparatoria
10,43,Mujer,Puebla,13865,Preparatoria


**anti_join() — Filtrado por diferencia**

Devuelve las filas de la izquierda que NO tienen coincidencia en la derecha.



In [20]:
d <-datos %>%
  anti_join(ciudades_filtradas, by = "ciudad")
#Muestra personas que viven en otras ciudades distintas a CDMX y Puebla.
head(d)

id,edad,sexo,ciudad,ingreso_mensual,nivel_educativo
<int>,<int>,<chr>,<chr>,<dbl>,<chr>
1,48,Hombre,Monterrey,20490,Universidad
3,31,Hombre,Guadalajara,18881,Posgrado
4,20,Mujer,Guadalajara,14925,Preparatoria
9,42,Mujer,Guadalajara,13675,Preparatoria
13,44,Mujer,Guadalajara,11072,Universidad
16,46,Mujer,Monterrey,18901,Posgrado


**replace_na() — Reemplazar NAs**

Ideal después de un join, para reemplazar valores vacíos con un valor por defecto:


In [25]:
d <- datos %>%
  left_join(ciudades_extra, by = "ciudad") %>%
  replace_na(list(region = "Desconocida"))
head(d)
#library(tidyr)  Para usar replace_na


id,edad,sexo,ciudad,ingreso_mensual,nivel_educativo,region
<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>
1,48,Hombre,Monterrey,20490,Universidad,Desconocida
2,32,Mujer,CDMX,12743,Posgrado,Centro
3,31,Hombre,Guadalajara,18881,Posgrado,Occidente
4,20,Mujer,Guadalajara,14925,Preparatoria,Occidente
5,59,Mujer,CDMX,16449,Secundaria,Centro
6,60,Mujer,Puebla,23045,Secundaria,Centro


**bind_rows() — Apilar tablas (uno debajo del otro)**

Útil cuando quieres combinar personas de dos fuentes distintas.

In [26]:
datos_nuevos <- tibble(
  id = 101:103,
  edad = c(22, 35, 50),
  sexo = c("Mujer", "Hombre", "Mujer"),
  ciudad = c("Toluca", "CDMX", "Puebla"),
  ingreso_mensual = c(14500, 16000, 13200),
  nivel_educativo = c("Universidad", "Posgrado", "Preparatoria")
)

# Combinar ambas bases
head(bind_rows(datos, datos_nuevos))


id,edad,sexo,ciudad,ingreso_mensual,nivel_educativo
<int>,<dbl>,<chr>,<chr>,<dbl>,<chr>
1,48,Hombre,Monterrey,20490,Universidad
2,32,Mujer,CDMX,12743,Posgrado
3,31,Hombre,Guadalajara,18881,Posgrado
4,20,Mujer,Guadalajara,14925,Preparatoria
5,59,Mujer,CDMX,16449,Secundaria
6,60,Mujer,Puebla,23045,Secundaria
